In [2]:
# 라이브러리 설치
!pip install python-docx pymupdf konlpy pandas openpyxl reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 24.4 MB/s eta 0:00:00


In [3]:
import xml.etree.ElementTree as ET
import docx
import fitz
import pandas as pd
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from google.colab import files
from konlpy.tag import Okt
from collections import defaultdict

In [4]:
okt = Okt()

In [5]:
def extract_hwpx(hwpx_file):
    try:
        tree = ET.parse(hwpx_file)
        root = tree.getroot()
        texts = [elem.text.strip() for elem in root.iter("t") if elem.text]
        return texts
    except Exception as e:
        return [f"오류 발생: {e}"]

def extract_docx(docx_file):
    try:
        doc = docx.Document(docx_file)
        texts = []

        # 본문 단락
        for para in doc.paragraphs:
            if para.text.strip():
                texts.append(para.text.strip())

        # 표 안의 텍스트
        for table in doc.tables:
            for row in table.rows:
                for cell in row.cells:
                    cell_text = cell.text.strip()
                    if cell_text:
                        texts.append(cell_text)

        return texts
    except Exception as e:
        return [f"오류 발생: {e}"]


def extract_pdf(pdf_file):
    try:
        doc = fitz.open(pdf_file)
        texts = []
        for page in doc:
            texts.extend([line.strip() for line in page.get_text("text").split("\n") if line.strip()])
        return texts
    except Exception as e:
        return [f"오류 발생: {e}"]

In [6]:
def extract_proper_nouns(text_list):
    proper_nouns = set()
    for text in text_list:
        tokens = okt.pos(text)
        for word, tag in tokens:
            if tag == 'Noun' and len(word) > 1:
                proper_nouns.add(word)
    return list(sorted(proper_nouns))

In [9]:
# 파일 업로드 및 텍스트 추출
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

if file_name.lower().endswith('.hwpx'):
    extracted_text = extract_hwpx(file_name)
elif file_name.lower().endswith('.docx'):
    extracted_text = extract_docx(file_name)
elif file_name.lower().endswith('.pdf'):
    extracted_text = extract_pdf(file_name)
else:
    extracted_text = ["지원되지 않는 파일 형식입니다."]

proper_nouns = extract_proper_nouns(extracted_text)

Saving 읍면동 단위 데이터 수집 모델 구축팀 4월 4일자 회의록.docx to 읍면동 단위 데이터 수집 모델 구축팀 4월 4일자 회의록 (1).docx


In [10]:
# 고유명사 추출 및 출력
def get_initial_consonant(word):
    if '가' <= word[0] <= '힣':
        cho_idx = (ord(word[0]) - ord('가')) // 588
        chosung_list = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ',
                        'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
        return chosung_list[cho_idx]
    else:
        return '기타'

grouped = defaultdict(list)
for noun in proper_nouns:
    cho = get_initial_consonant(noun)
    grouped[cho].append(noun)

# DataFrame으로 변환
grouped_df = pd.DataFrame({
    'Proper Nouns': {k: ', '.join(sorted(v)) for k, v in grouped.items()}
}).sort_index()

# 출력
print(grouped_df)

                                        Proper Nouns
ㄱ   각각, 검토, 결과, 결과물, 결정, 계정, 고민, 과업, 관리, 교수, 구축, 금요일
ㄴ                                             내용, 논의
ㄷ                                다음, 단위, 대한, 대해, 데이터
ㄹ                                                 리기
ㅁ               만날, 만남, 메타, 멘토링, 모델, 목요일, 목표, 문의, 미팅
ㅂ                                 반영, 발표, 방법, 방안, 불참
ㅅ                   사용자, 사항, 선택, 설명, 수준, 수집, 시간, 시스템
ㅇ  어디, 업로드, 예정, 오전, 오프라인, 온라인, 완성, 요일, 월요일, 위치, 유...
ㅈ                            자동화, 작업, 정혁, 지적, 진행, 질문
ㅊ                                차수, 참석자, 처리, 첨부, 추가
ㅌ                                            태그, 토요일
ㅍ                                                 필요
ㅎ                                                 회의


In [ ]:
# 사용자 선택에 따라 저장
excel_choice = input("엑셀 파일로 저장하시겠습니까? (y/n): ").strip().lower()
if excel_choice == 'y':
    df = pd.DataFrame(grouped_df, columns=["Proper Nouns"])
    df.to_excel("grouped_df.xlsx", index=False)
    print("✔ grouped_df.xlsx 파일로 저장되었습니다.")

csv_choice = input("CSV 파일로 저장하시겠습니까? (y/n): ").strip().lower()
if csv_choice == 'y':
    df = pd.DataFrame(grouped_df, columns=["Proper Nouns"])
    df.to_csv("grouped_df.csv", index=False, encoding='utf-8-sig')
    print("✔ grouped_df.csv 파일로 저장되었습니다.")

pdf_choice = input("PDF 파일로 저장하시겠습니까? (y/n): ").strip().lower()
if pdf_choice == 'y':
    pdfmetrics.registerFont(TTFont('NanumGothic', '/content/NanumGothic.ttf'))
    c = canvas.Canvas("grouped_df.pdf", pagesize=A4)
    textobject = c.beginText(40, 800)
    textobject.setFont("NanumGothic", 12)

    for noun in grouped_df:
        textobject.textLine(noun)

    c.drawText(textobject)
    c.save()
    print("✔ grouped_df.pdf 파일로 저장되었습니다.")


엑셀 파일로 저장하시겠습니까? (y/n): n
CSV 파일로 저장하시겠습니까? (y/n): y
✔ grouped_df.csv 파일로 저장되었습니다.
PDF 파일로 저장하시겠습니까? (y/n): n
